In [12]:
import pickle
import pandas as pd
import xgboost as xgb

In [13]:
X = pd.read_feather('/kaggle/input/amex-imputed-and-1hot-encoded/X_train.ftr') 
y = pd.read_feather('/kaggle/input/amex-imputed-and-1hot-encoded/y_train.ftr')  
X = X.set_index('customer_ID')
y = y.set_index('customer_ID')['target']

In [14]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [15]:
model = xgb.XGBClassifier() #create XGB Classifier model with default parameters


In [16]:
model = model.fit(X,y) #fit model with aggregate training data

In [18]:
preds = model.predict(X)
print(amex_metric(pd.read_feather('../input/amex-imputed-and-1hot-encoded/y_train.ftr').set_index('customer_ID'), pd.DataFrame(preds, index=y.index, columns=['prediction'])))

0.5165340445639233


In [19]:
from sklearn.metrics import classification_report

In [20]:
print(classification_report(y, preds))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92   4153582
           1       0.77      0.77      0.77   1377869

    accuracy                           0.88   5531451
   macro avg       0.84      0.84      0.84   5531451
weighted avg       0.88      0.88      0.88   5531451



In [22]:
pickle.dump(model, open('xgb_Classifier_model_feather.sav', 'wb'))